In [ ]:
import earthaccess
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

auth = earthaccess.login()
bounds = (-120.75, 33.5, -119, 34.5) 


df_kelp = pd.read_csv("../../data/PISCO_kelpdata.csv")
df_kelp['time'] = pd.to_datetime(df_kelp[['year', 'month', 'day']])
sst_query = earthaccess.search_data(
    short_name="MUR-JPL-L4-GLOB-v4.1",
    bounding_box=bounds,
    temporal=("2024-01-01", "2024-12-31")
)
ds_sst = xr.open_mfdataset(earthaccess.open(sst_query), chunks={'time': 30}, engine="h5netcdf")
sst_c = ds_sst.analysed_sst - 273.15
ds_cuti = xr.open_dataset("your_cuti_dailies.nc")


def get_env(row):
    try:
        s = sst_c.sel(lat=row.latitude, lon=row.longitude, time=row.time, method='nearest').values
        c = ds_cuti.cuti.sel(lat=row.latitude, time=row.time, method='nearest').values
        return pd.Series([float(s), float(c)])
    except:
        return pd.Series([np.nan, np.nan])


df_kelp[['sst', 'cuti']] = df_kelp.apply(get_env, axis=1)
df_final = df_kelp.dropna(subset=['sst', 'cuti', 'observation_count']).sort_values('time')


r_sst, _ = pearsonr(df_final['sst'], df_final['observation_count'])
r_cuti, _ = pearsonr(df_final['cuti'], df_final['observation_count'])


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot A: SST vs Kelp
sns.regplot(data=df_final, x='sst', y='observation_count', ax=ax1, 
            scatter_kws={'alpha':0.4}, line_kws={'color':'red'})
ax1.set_title(f"Thermal Stress: SST vs Kelp\n(Pearson r = {r_sst:.2f})")
ax1.set_xlabel("SST (°C)")
ax1.set_ylabel("In Situ Kelp Count")

# Plot B: CUTI vs Kelp
sns.regplot(data=df_final, x='cuti', y='observation_count', ax=ax2, 
            scatter_kws={'alpha':0.4, 'color':'green'}, line_kws={'color':'darkgreen'})
ax2.set_title(f"Nutrient Supply: CUTI vs Kelp\n(Pearson r = {r_cuti:.2f})")
ax2.set_xlabel("Upwelling Index (CUTI)")
ax2.set_ylabel("In Situ Kelp Count")

plt.tight_layout()
plt.savefig("kelp.png")
plt.show()
df_final.to_csv("kelp.csv", index=False)


Enter your Earthdata Login username:  chasante
Enter your Earthdata password:  ········


QUEUEING TASKS | :   0%|          | 0/367 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/367 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/367 [00:00<?, ?it/s]

/tmp/ipykernel_5436/2641847501.py:20: FutureWarning: In a future version, xarray will not decode the variable 'dt_1km_data' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  ds_sst = xr.open_mfdataset(earthaccess.open(sst_query), chunks={'time': 30}, engine="h5netcdf")
/tmp/ipykernel_5436/2641847501.py:20: FutureWarning: In a future version, xarray will not decode the variable 'dt_1km_data' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which 